In [31]:
import os
import json
import pandas as pd
import requests
import time
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from urllib.parse import quote_plus
from bs4 import BeautifulSoup, Comment, Doctype
import re
import time

load_dotenv()

True

In [32]:
# Credenciales de la Base de Datos ANTIGUA (saingo_v2)
DB_OLD_USER = "root"
DB_OLD_PASS = ""
DB_OLD_HOST = "127.0.0.1"
DB_OLD_NAME = "sai_v2"

# Credenciales de la Base de Datos NUEVA (saingo_v3)
DB_NEW_USER = "root"
DB_NEW_PASS = ""
DB_NEW_HOST = "127.0.0.1"
DB_NEW_NAME = "sai_v5"

ID_MAPPING = {
    29: 71,
    20: 70,
    17: 69,
    16: 68,
    15: 67,
    14: 66,
    13: 65,
    12: 64,
    7: 63
}

REPORTS_STATIC_CONTENT_STRUCTURE = {
    "en": {
        # Puedes añadir campos aquí si los necesitas, ej: "author_bio", "related_links_title"
        "main_banner_title": "News & Updates",
        "main_banner_subtitle": "Stay informed about our latest activities and impact.",
    },
    "es": {
        "main_banner_title": "Noticias y Actualizaciones",
        "main_banner_subtitle": "Mantente informado sobre nuestras últimas actividades e impacto.",
    }
}

# Convertimos la estructura a un string JSON una sola vez para reutilizarlo.
REPORTS_STATIC_CONTENT_JSON = json.dumps(REPORTS_STATIC_CONTENT_STRUCTURE, ensure_ascii=False)

In [33]:
def get_post_translations(post_id, engine):
    """Obtiene todas las traducciones para un ID de blog_post y las devuelve como un diccionario."""
    query = text("""
        SELECT column_name, value 
        FROM translations 
        WHERE table_name = 'blog_posts' AND foreign_key = :post_id AND locale = 'es'
    """)
    translations = {}
    try:
        with engine.connect() as connection:
            result = connection.execute(query, {'post_id': post_id})
            for row in result:
                translations[row.column_name] = row.value
    except Exception as e:
        print(f"⚠️  No se pudieron obtener traducciones para el post ID {post_id}: {e}")
    return translations



def call_azure_openai(system_prompt, user_message):
    """Envía una solicitud a Azure OpenAI y devuelve la respuesta."""
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key = os.getenv("OPENAI_API_KEY")
    
    if not endpoint or not api_key:
        print("❌ Faltan credenciales de Azure OpenAI en el archivo .env")
        return None
    
    headers = {"Content-Type": "application/json", "api-key": api_key}
    payload = {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ],
        "max_tokens": 4096,
        "temperature": 0.5,
    }
    
    for attempt in range(5):  # hasta 5 intentos
        try:
            response = requests.post(endpoint, headers=headers, json=payload, timeout=90)
            if response.status_code == 429:
                wait_time = 2 ** attempt  # espera creciente: 1s, 2s, 4s, 8s, 16s
                print(f"🔁 Esperando {wait_time} segundos por límite de tasa (429)...")
                time.sleep(wait_time)
                continue
            elif response.status_code != 200:
                print(f"❌ Error en API de Azure {response.status_code}: {response.text}")
                return None
            return response.json()["choices"][0]["message"]["content"]
        except requests.exceptions.RequestException as e:
            print(f"❌ Error en la solicitud a Azure: {e}")
            return None
        except (KeyError, IndexError, json.JSONDecodeError) as e:
            print(f"❌ Error procesando respuesta de Azure: {e}")
            return None

def simplify_html(html_content):
    """Simplifica el HTML eliminando scripts, estilos y atributos innecesarios."""
    if not html_content or pd.isna(html_content):
        return ""
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        for element in soup(['script', 'style', 'meta', 'link', 'head', 'noscript', 'iframe']):
            element.decompose()
        for tag in soup.find_all(True):
            allowed_attrs = ['href', 'src'] # Conservar solo href para links y src para imágenes
            attrs = {attr: tag.attrs[attr] for attr in allowed_attrs if attr in tag.attrs}
            tag.attrs = attrs
        return str(soup)
    except Exception as e:
        print(f"⚠️ Error simplificando HTML: {e}")
        return html_content[:8000]

def create_db_engine(user, password, host, db_name):
    """Crea un motor de conexión SQLAlchemy."""
    password_encoded = quote_plus(password)
    connection_string = f"mysql+mysqlconnector://{user}:{password_encoded}@{host}/{db_name}"
    return create_engine(connection_string)

def load_prompt_template(filepath="promptreport.txt"):
    """Carga la plantilla del prompt desde un archivo."""
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.read()

# --- 3. FASES DE MIGRACIÓN PARA REPORTS ---

# --- FASE 1: EXTRACCIÓN ---
# def extract_blog_posts():
#     """Extrae los datos de la tabla 'blog_posts' de la BD antigua."""
#     print("Iniciando Fase 1: Extracción de 'blog_posts'...")
#     engine_old = create_db_engine(DB_OLD_USER, DB_OLD_PASS, DB_OLD_HOST, DB_OLD_NAME)
    
#     query = """
#     SELECT 
#         id, title, excerpt, slug, body, seo_title, meta_description, tags, campaign_id
#     FROM blog_posts;
#     """
#     try:
#         df = pd.read_sql(query, engine_old)
#         print(f"✅ Extracción completada. Se encontraron {len(df)} registros.")
#         return df
#     except Exception as e:
#         print(f"❌ Error al extraer datos de 'blog_posts': {e}")
#         return pd.DataFrame()

def extract_blog_posts():
    """Extrae datos de 'blog_posts' y AÑADE las traducciones existentes desde 'translations'."""
    print("Iniciando Fase 1: Extracción de 'blog_posts' y sus traducciones...")
    engine_old = create_db_engine(DB_OLD_USER, DB_OLD_PASS, DB_OLD_HOST, DB_OLD_NAME)
    
    # La consulta a 'blog_posts' sigue siendo la misma
    query = """
    SELECT 
        id, title, excerpt, slug, body, seo_title, meta_description, tags, campaign_id
    FROM blog_posts;
    """
    try:
        df = pd.read_sql(query, engine_old)
        print(f"✅ Extracción de 'blog_posts' completada. Se encontraron {len(df)} registros.")

        # NUEVO: Obtener traducciones para cada post y añadirlas al DataFrame
        translations_list = [get_post_translations(post_id, engine_old) for post_id in df['id']]
        df['translations_es'] = translations_list
        
        print("✅ Traducciones asociadas correctamente.")
        return df
    except Exception as e:
        print(f"❌ Error al extraer datos de 'blog_posts': {e}")
        return pd.DataFrame()

# --- FASE 2: TRANSFORMACIÓN ---
# def process_body_with_ai(html_content, prompt_template):
#     """Procesa el contenido HTML con Azure OpenAI para obtener un JSON."""
#     if not html_content or pd.isna(html_content):
#         return json.dumps({"en": "", "es": ""})
    
#     simplified_html = simplify_html(html_content)
#     orig_size = len(html_content) if html_content else 0
#     new_size = len(simplified_html)
#     print(f"  Reducción HTML: {orig_size} → {new_size} caracteres")
    
#     system_prompt = prompt_template.split("---")[0].strip()
#     user_message = f"CONTENIDO HTML A PROCESAR:\n{simplified_html}"
    
#     response_content = call_azure_openai(system_prompt, user_message)
    
#     if response_content is None:
#         error_msg = "No se pudo obtener respuesta de Azure OpenAI"
#         return json.dumps({"en": error_msg, "es": error_msg})
    
#     try:
#         cleaned_response = response_content.strip().replace("```json", "").replace("```", "")
#         parsed_json = json.loads(cleaned_response)
#         return json.dumps(parsed_json, ensure_ascii=False)
#     except Exception as e:
#         print(f"⚠️ Error al parsear JSON de la IA: {e}")
#         return json.dumps({"en": "Error al parsear respuesta", "es": "Error al parsear respuesta"})

# def transform_reports_data(df):
#     """Transforma el DataFrame de blog_posts al formato de la nueva tabla 'reports'."""
#     print("\nIniciando Fase 2: Transformación de datos para 'reports'...")
#     if df.empty:
#         return []
    
#     df = df.iloc[0:1]

#     prompt_template = load_prompt_template()
#     transformed_records = []
    
#     for index, row in df.iterrows():
#         print(f"  Procesando registro {index + 1}/{len(df)} (ID antiguo: {row['id']})...")
        

#         old_campaign_id = row.get('campaign_id')
#         project_id_to_assign = ID_MAPPING.get(old_campaign_id, None)

#         if project_id_to_assign is None:
#             print(f"  ⚠️ Advertencia: No se encontró mapeo para campaign_id {old_campaign_id}")
#             # Opción 1: Asignar un valor por defecto (ej. 1)
#             project_id_to_assign = 1

#         text_editor_content_json = process_body_with_ai(row['body'], prompt_template)
        
#         # Construir el JSON para la columna 'meta'
#         meta_en = {
#             "seo_title": row.get('seo_title'),
#             "seo_description": row.get('meta_description'),
#             "keywords": row.get('tags'),
#             "og_title": row.get('seo_title'), # Reutilizamos seo_title o puedes dejarlo null
#             "og_description": row.get('meta_description')
#         }
#         # Dejamos 'es' vacío para ser llenado después si es necesario
#         meta_es = {"seo_title": None, "seo_description": None, "keywords": None, "og_title": None, "og_description": None}
        
#         # Mapear los campos al nuevo formato
#         record = {
#             "project_id": project_id_to_assign,
#             "slug": json.dumps({"en": row.get('slug', ''), "es": row.get('slug', '')}, ensure_ascii=False),
#             "title": json.dumps({"en": row.get('title', ''), "es": ""}, ensure_ascii=False),
#             "excerpt": json.dumps({"en": row.get('excerpt', ''), "es": ""}, ensure_ascii=False),
#             "content": STATIC_CONTENT_JSON,
#             "text_editor_content": text_editor_content_json, # Contenido de la IA aquí
#             "meta": json.dumps({"en": meta_en, "es": meta_es}, ensure_ascii=False),
#             "status": "published",
#             "creator_id": 1, # ¡IMPORTANTE! Asigna el ID del usuario creador correcto
#             "editor_id": 1,  # ¡IMPORTANTE! Asigna el ID del usuario editor correcto
#         }
#         transformed_records.append(record)
        
#     print("✅ Transformación completada.")
#     return transformed_records

# --- FASE 2: TRANSFORMACIÓN (MODIFICADA) ---

def process_body_with_ai(body_en, body_es, prompt_template):
    """Procesa el contenido HTML bilingüe con Azure OpenAI."""
    # Si ambos cuerpos están vacíos, no llamar a la IA
    if not body_en and not body_es:
        return json.dumps({"text_editor_content": {"en": "", "es": ""}})

    # Simplificar HTML para ambos idiomas
    simplified_en = simplify_html(body_en)
    simplified_es = simplify_html(body_es)
    
    # El mensaje de usuario ahora es un JSON con ambos idiomas
    user_message = json.dumps({
        "en": simplified_en,
        "es": simplified_es
    }, ensure_ascii=False)
    
    system_prompt = prompt_template.split("---")[0].strip()
    
    response_content = call_azure_openai(system_prompt, user_message)
    
    if response_content is None:
        error_msg = "No se pudo obtener respuesta de Azure OpenAI"
        return json.dumps({"text_editor_content": {"en": error_msg, "es": error_msg}})
    
    try:
        # La IA ya debería devolver un JSON, solo lo limpiamos por si acaso
        cleaned_response = response_content.strip().replace("```json", "").replace("```", "")
        parsed_json = json.loads(cleaned_response)
        # La IA devuelve un objeto con la clave 'text_editor_content', lo devolvemos tal cual
        return json.dumps(parsed_json.get('text_editor_content', {}), ensure_ascii=False)
    except Exception as e:
        print(f"⚠️ Error al parsear JSON de la IA: {e}")
        return json.dumps({"en": "Error al parsear respuesta", "es": "Error al parsear respuesta"})

def transform_reports_data(df):
    """Transforma el DataFrame de blog_posts al formato de 'reports' usando traducciones."""
    print("\nIniciando Fase 2: Transformación de datos para 'reports'...")
    if df.empty:
        return []
    
    # Limita el procesamiento a 1 registro para pruebas, puedes comentarlo o quitarlo
    df = df.iloc[1:2]

    prompt_template = load_prompt_template()
    transformed_records = []
    
    for index, row in df.iterrows():
        print(f"  Procesando registro {index + 1}/{len(df)} (ID antiguo: {row['id']})...")
        
        # Obtenemos el diccionario de traducciones
        translations = row.get('translations_es', {})

        # Mapeo de campaign_id a project_id (esta lógica se mantiene)
        old_campaign_id = row.get('campaign_id')
        project_id_to_assign = ID_MAPPING.get(old_campaign_id,1)
        #project_id_to_assign = 1
        if project_id_to_assign is None:
            print(f"  ⚠️ Advertencia: No se encontró mapeo para campaign_id {old_campaign_id}. Asignando 'None'.")
        
        # MODIFICADO: Llamada a la IA con ambos cuerpos de texto
        text_editor_content_json = process_body_with_ai(
            row.get('body'), 
            translations.get('body', row.get('body')), # Fallback al inglés si no hay traducción del body
            prompt_template
        )
        
        # MODIFICADO: Construir el JSON para la columna 'meta' con datos bilingües
        meta_en = {
                "seo_title": row.get('seo_title'),
            "seo_description": row.get('meta_description'),
            "keywords": row.get('tags'),
            "og_title": row.get('seo_title'),
            "og_description": row.get('meta_description')
        }
        meta_es = {
            "seo_title": translations.get('seo_title'),
            "seo_description": translations.get('meta_description'),
            "keywords": translations.get('tags'),
            "og_title": translations.get('seo_title'),
            "og_description": translations.get('meta_description')
        }
        
        record = {
            "project_id": project_id_to_assign,
            "slug": json.dumps({
                "en": row.get('slug', ''), 
                "es": translations.get('slug', row.get('slug', ''))
            }, ensure_ascii=False),
            "title": json.dumps({
                "en": row.get('title', ''), 
                "es": translations.get('title', row.get('title', ''))
            }, ensure_ascii=False),
            "excerpt": json.dumps({
                "en": row.get('excerpt', ''), 
                "es": translations.get('excerpt', row.get('excerpt', ''))
            }, ensure_ascii=False),
            "content": REPORTS_STATIC_CONTENT_JSON, # Usamos la nueva plantilla
            "text_editor_content": text_editor_content_json,
            "meta": json.dumps({"en": meta_en, "es": meta_es}, ensure_ascii=False),
            "status": "published",
            "creator_id": 1,
            "editor_id": 1,
        }
        transformed_records.append(record)
        
    print("✅ Transformación completada.")
    return transformed_records

# --- FASE 3: CARGA ---
def load_into_reports_table(records):
    """Carga los registros transformados en la tabla 'reports'."""
    print("\nIniciando Fase 3: Carga de datos en 'reports'...")
    if not records:
        print("No hay registros para cargar.")
        return

    engine_new = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME)
    
    insert_query = text("""
        INSERT INTO reports (
            project_id, slug, title, excerpt, content, text_editor_content, meta,
            status, creator_id, editor_id, created_at, updated_at
        ) VALUES (
            :project_id, :slug, :title, :excerpt, :content, :text_editor_content, :meta,
            :status, :creator_id, :editor_id, NOW(), NOW()
        )
    """)
    
    try:
        with engine_new.connect() as connection:
            for record in records:
                connection.execute(insert_query, record)
            connection.commit()
        print(f"✅ Carga completada. Se han insertado {len(records)} registros en 'reports'.")
    except Exception as e:
        print(f"❌ Error al cargar datos en 'reports': {e}")


In [34]:
print("🚀 Iniciando script de migración de 'blog_posts' a 'reports'.")
    
# Paso 1: Extraer
blog_posts_df = extract_blog_posts()
    
# Paso 2: Transformar
if not blog_posts_df.empty:
    reports_data = transform_reports_data(blog_posts_df)
        
    # Paso 3: Cargar
    load_into_reports_table(reports_data)
    
print("\n🎉 Proceso de migración de reportes finalizado.")

🚀 Iniciando script de migración de 'blog_posts' a 'reports'.
Iniciando Fase 1: Extracción de 'blog_posts' y sus traducciones...
✅ Extracción de 'blog_posts' completada. Se encontraron 87 registros.
✅ Traducciones asociadas correctamente.

Iniciando Fase 2: Transformación de datos para 'reports'...
  Procesando registro 2/1 (ID antiguo: 4)...
✅ Transformación completada.

Iniciando Fase 3: Carga de datos en 'reports'...
❌ Error al cargar datos en 'reports': (mysql.connector.errors.IntegrityError) 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`sai_v5`.`reports`, CONSTRAINT `reports_project_id_foreign` FOREIGN KEY (`project_id`) REFERENCES `projects` (`id`) ON DELETE CASCADE)
[SQL: 
        INSERT INTO reports (
            project_id, slug, title, excerpt, content, text_editor_content, meta,
            status, creator_id, editor_id, created_at, updated_at
        ) VALUES (
            %(project_id)s, %(slug)s, %(title)s, %(excerpt)s, %(content)s, %(tex